# VOI Locator for uCT Imagery

This is effectively a simple adaptation of the typical fast normalised cross correlation algorithm (published by (]J. P. Lewis in 2005) from a single slice into a three dimensional stack. We have a series of stacks with a sedimenting material which we can work through in one of two ways. Either we can analyse the whole lot in a single shot using a nested for loop or simply go one stack at a time with a single loop. To run this script we need the scikit image `skimage` `io` submodule and the `match_template` function from the `feature` submodule. We also include `matplotlib` for debugging and good measure.`glob` is required for reading directories from the filesystem. 

To test the code we can apply the analysis back to the first stack and should get a regression coefficient $R = 1.0$.

This script has been tested working on Python 3.6 on Arch Linux and Microsoft Windows 10 operating systems prior to publication with scikit-image 0.16.1

TODO:

-- Test with scikit-image 0.17.1 (05-2020).

-- increase speed of analysis by defining a trigger threshold for the regression coefficient at which we exit the loop.

-- Increase speed of analysis by predictive locating.

-- Code for single shot approach detailed above.

In [3]:
from skimage import io
from skimage.feature import match_template
import matplotlib.pyplot as plt
import numpy as no
%matplotlib inline
import glob

Though we typically reconstruct tomographic data into a 16-bit or 32-bit image series, I've lowered the precision here as all we are interested in is the internal morphology and a 16-bit precision is unnecessary and just eats disk space. If we want we can load a Zeiss txrm file using `tomopy` but I've not shown this here as it will require the file path and for loop to be changed.

In [ ]:
stack = sorted(glob.glob("8-bit/*.tiff"))
# For tif, jpg, and png files respectively, a simple change.
#stack = sorted(glob.glob("*.tif"))
#stack = sorted(glob.glob("8-bit/*.jpg"))
#stack = sorted(glob.glob("8-bit/*.png"))

We need to load a slice which represents our first VOI. This would normally be taken from the first scan, but there's nothing stopping you from taking from the last scan and working forwards. `VOI` is the  slice representative of the Volume of Interest. Again, change the file extension similarly to the above cell if required. I've not counted the number of slices here for abstraction.

In [ ]:
VOI_slice = "VOISlice.tiff"

Here, we begin iterating through the stack. `interog_slice` is the slice in the stack we are interrogating at this point in the loop. We load each slice into this numpy array using imread from `skimage.io`. 

Note: It is possible here to load the entire stack into RAM and run through the whole thing in a single hit. This isn't necessarily any quicker if you're on an SSD from what I can tell. 

In `results` we store our maximum coefficient $R$ for each slice. $x$ and $y$ are written to columns 0 and 1, and $R$ to column 2. The $z$ value is the argument to the maximum for $R$, and then we can take $x$ and $y$ back from the array.

In [ ]:
values = np.empty((len(stack), 3))

for ii in range(0, len(stack)):
    # Load the interrogated slice (see above).
    interog_slice = io.imread(stack[ii])
    
    # Run the match_template fast normalised cross correlation algorithm.
    result = match_template(interog_slice, VOI_slice)
    
    # Convert the flat argmax into a tuple of xy values
    ab = np.unravel_index(np.argmax(result), result.shape)
    
    # Write the x, y, and R results to the values array
    values[ii,2] = np.max(result)
    x,y = ab[::-1]
    values[ii,0] = x
    values[ii,1] = y
    
# The z location is the argument to the maximum in column 0 (see above)
z = np.argmax(values[:,0])

# Then x and y are the values at this maximum in columns 1 and 2
x = values[z,1]
y = values[z,2]

# Finish with some nice output
print("VOI Located at x = {}, y = {}, z = {}".format(x,y,z))

Unfinished and untested ... for the trigger threshold approach, try:

If we hit a value that is greater than `trigger_threshond` we start watching for increasing $R$ values until the rate of increase becomes less than `trigger_delta` at which point $R$ is probably about to reduce as we drop through the stack and we can exit the loop.

In [ ]:
previous R = 0
trigger_delta = 0.1
trigger = 0.95

See commented code...

In [ ]:
values = np.empty((len(stack), 3))

for ii in range(0, len(stack)):
    # Load the interrogated slice (see above).
    interog_slice = io.imread(stack[ii])
    
    # Run the match_template fast normalised cross correlation algorithm.
    result = match_template(interog_slice, VOI_slice)
    
    # Convert the flat argmax into a tuple of xy values
    ab = np.unravel_index(np.argmax(result), result.shape)
    
    # Write the x, y, and R results to the values array.
    # This may seem like poor abstraction but it makes sense for readability...
    R = np.max(result)
    x,y = ab[::-1]
    
    values[ii,2] = R
    values[ii,0] = x
    values[ii,1] = y
    
    # Test R to see if it is above the trigger value
    if (R > trigger):
        # If the difference between the previous R value and this R value
        # is greater than the delta value our numbers are still changing rapidly
        # so set the previous_R value to R and continue the loop.
        if ((R - previous_R) > trigger_delta)
            previous_R = R
        # The change in R has dropped below the triggering delta value,
        # so we can break the loop and write the value to output.
        elif ((R - previous_R < trigger_delta))
            break;
        
# The z location is the argument to the maximum in column 0 (see above)
z = np.argmax(values[:,0])

# Then x and y are the values at this maximum in columns 1 and 2
x = values[z,1]
y = values[z,2]

# Finish with some nice output
print("VOI Located at x = {}, y = {}, z = {}".format(x,y,z))